In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [3]:
## change ##
batch_size = 16
clip = .25

Prepare data

In [4]:
train_gen = SequenceGenSpec(train)
#train_gen = mixupWrapper(SequenceGenSpec(train),4e-2) # does not improve
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [23]:
convNets = Sequential(Dropout(0.5),
                      ResidualBlockGLU(129),
                      Dropout(0.5),
                      ResidualBlockGLU(258),
                      Dropout(0.5),
                      GRU_NCL(516,516,2,batch_first=True,dropout=0.5,bidirectional=True,returnH=True))

In [24]:
linear = Sequential(Dropout(0.5),Linear(516*4,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [25]:
model = CNN_RNN2seq(convNets,linear).to('cuda:0')

In [26]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [27]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [28]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.7186041955764475, val_loss:2.7511844635009766
epoch:1, train_loss:2.380261280788825, val_loss:2.5895111560821533
epoch:2, train_loss:2.300448706516853, val_loss:2.598405361175537
epoch:3, train_loss:2.27506344134991, val_loss:2.498331069946289
epoch:4, train_loss:2.188895743626815, val_loss:2.45334792137146
Training completed in 87.80019021034241s
